In [1]:
import pandas as pd
### use xarray for extracting temperature data from .nc files
import xarray as xr 
import numpy as np
import geopandas as gpd
import datetime
import os
import metpy.calc as mpcalc
from metpy.units import units
import rioxarray
import zipfile
import os
# Directory to extract files
extract_dir = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/united_states/extracted'

#Geometry of the United States
gdf = gpd.read_file('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/adm0_shp/united_states/clean_shp.shp')

## Preprocess

In [95]:

# Ensure the directory exists
os.makedirs(extract_dir, exist_ok=True)


# Unzip the NetCDF files
for year in ['2018', '2019', '2020', '2021', '2022']:
    zip_path = f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/united_states/unextracted/main/era5_land_dewpoint_us_{year}.netcdf.zip'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
                # Rename extracted files
        for file_name in zip_ref.namelist():
            if file_name.endswith('.nc'):
                old_file_path = os.path.join(extract_dir, file_name)
                new_file_path = os.path.join(extract_dir, f'era5_land_dewpoint_us_{year}.nc')
                os.rename(old_file_path, new_file_path)

# Unzip leftover_months NetCDF files
for month in ['04', '10']:
    for year in ['2018', '2019', '2020', '2021', '2022']:
        zip_path = f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/united_states/unextracted/{month}/era5_land_dewpoint_us_{year}_{month}.netcdf.zip'
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
                    # Rename extracted files
            for file_name in zip_ref.namelist():
                if file_name.endswith('.nc'):
                    old_file_path = os.path.join(extract_dir, file_name)
                    new_file_path = os.path.join(extract_dir, f'era5_land_dewpoint_us_{year}_{month}.nc')
                    os.rename(old_file_path, new_file_path)            



### Step 1: Get daily maximum THI values at each pixel cell

In [ ]:


#Concatenate data from all 5 years along the time axis
#Change the time zone
#Convert temperature and dewpoint to Celcius
#Calculate relative humidity using metpy package
#Calculate heat index using metpy

# Resample to get the DAILY MAXIMUM heat index value at each pixel cell -- 'concat_daily_max'



In [ ]:
data_arrays = []

for year in ['2018', '2019', '2020', '2021', '2022']:

    #May to September files
    nc_file = os.path.join(extract_dir, f'era5_land_dewpoint_us_{year}.nc')
    ds_main = xr.open_dataset(nc_file)

    #Extra April day
    nc_file_04 = os.path.join(extract_dir, f'era5_land_dewpoint_us_{year}_04.nc')
    ds_04 = xr.open_dataset(nc_file_04)

    #extra October day
    nc_file_10 = os.path.join(extract_dir, f'era5_land_dewpoint_us_{year}_10.nc')
    ds_10 = xr.open_dataset(nc_file_10)

    #concatenate
    ds = xr.concat([ds_main, ds_04, ds_10], dim='valid_time')
    ds = ds.sortby('valid_time')

    # Rename 'valid_time' to 'time'
    ds = ds.rename({'valid_time': 'time'})
    
    # Convert time to the desired timezone (e.g., 'America/Denver')
    ds['time'] = ds['time']- pd.Timedelta(hours=6)

    # Convert from Kelvin to Celsius
    ds['t2m'] = ds['t2m'] - 273.15
    ds['d2m'] = ds['d2m'] - 273.15

    # Calculate relative humidity
    ds['rh'] = mpcalc.relative_humidity_from_dewpoint(ds['t2m'] * units.degC, ds['d2m'] * units.degC)
    ds['rh'] = ds['rh'] * 100

    # Calculate THI using metpy's heat_index function
    ds['thi'] = mpcalc.heat_index(ds['t2m'] * units.degC, ds['rh'] * units.percent, mask_undefined=False)


    daily_max_thi = ds.resample(time='1D').max()

    data_arrays.append(daily_max_thi)

    print('appended')

    
concat_daily_max = xr.concat(data_arrays, dim='time')
concat_daily_max.to_netcdf('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/united_states/concat_array.nc')

In [ ]:
#Add coordinate projection to raster file

In [3]:
concat_daily_max = xr.open_dataset('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/united_states/array/concat_array.nc')
concat_daily_max.rio.write_crs(4326, inplace=True)

<xarray.Dataset> Size: 3GB
Dimensions:      (latitude: 270, longitude: 671, time: 785)
Coordinates:
    number       int64 8B ...
  * latitude     (latitude) float64 2kB 50.22 50.12 50.02 ... 23.52 23.42 23.32
  * longitude    (longitude) float64 5kB -126.5 -126.4 -126.3 ... -59.57 -59.47
  * time         (time) datetime64[ns] 6kB 2018-04-29 2018-04-30 ... 2022-10-02
    spatial_ref  int64 8B 0
Data variables:
    t2m          (time, latitude, longitude) float32 569MB ...
    d2m          (time, latitude, longitude) float32 569MB ...
    rh           (time, latitude, longitude) float32 569MB ...
    thi          (time, latitude, longitude) float64 1GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-12T23:47 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# For each location in GeoDataFrame of country
# Clip all pixel cells with intersect with shape of the location
# Get the mean value at each time period (the daily maximums calculated in step 1)
# Output is the mean of the daily maximums on each day for each county
# saved into a dictionary

### Step 2: Clip all pixels with intersect with each location, get the mean of those daily maximum values

In [4]:

data_dic = {}

for idx, row in gdf.iterrows():

    adm2 = row['adm2']
    adm1 = row['adm1']
    # Clip the THI DataArray using the geometry
    try:
        clipped_thi = concat_daily_max.rio.clip([row.geometry], all_touched=True, crs=gdf.crs)
        
        # Convert the clipped THI DataArray to a DataFrame
        clipped_thi_df = clipped_thi.to_dataframe().reset_index()
        
        # Add the adm2 information to the DataFrame

        clipped_thi_df = clipped_thi_df.groupby('time').mean(numeric_only=True).reset_index()

        clipped_thi_df['adm2'] = adm2
        
        clipped_thi_df['adm1'] = adm1

        data_dic[f'{adm1}_{adm2}'] = clipped_thi_df

        print(f'{adm1}_{adm2} appended')

    except:
        continue

    


Minnesota_Lake of the Woods appended
Washington_Ferry appended
Washington_Stevens appended
Washington_Okanogan appended
Washington_Pend Oreille appended
Idaho_Boundary appended
Montana_Lincoln appended
Montana_Flathead appended
Montana_Glacier appended
Montana_Toole appended
Montana_Liberty appended
Montana_Hill appended
Montana_Sheridan appended
North Dakota_Divide appended
North Dakota_Burke appended
North Dakota_Renville appended
North Dakota_Bottineau appended
North Dakota_Rolette appended
North Dakota_Towner appended
North Dakota_Pembina appended
Minnesota_Kittson appended
Minnesota_Roseau appended
Montana_Blaine appended
Montana_Phillips appended
Montana_Valley appended


In [ ]:
#Convert dictionary to dataframe

In [13]:
final_concat = pd.concat(data_dic.values(), ignore_index=True)

In [26]:
final_concat.drop(columns=['latitude', 'longitude'], inplace=True)

In [ ]:
#Save as file

In [24]:
final_concat.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/united_states/daily_max_THI.csv')

In [19]:
df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/united_states/daily_max_THI.csv')